![Neptune + XGBoost](https://neptune.ai/wp-content/uploads/2023/09/xgb.svg)

# Neptune + XGBoost

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/examples/blob/main/integrations-and-supported-tools/xgboost/notebooks/Neptune_XGBoost.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a><a target="_blank" href="https://github.com/neptune-ai/examples/blob/main/integrations-and-supported-tools/xgboost/notebooks/Neptune_XGBoost.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a><a target="_blank" href="https://app.neptune.ai/o/common/org/xgboost-integration/runs/details?viewId=5aec39d8-8e09-4be7-ae02-e17768b4fcb5&detailsTab=metadata&shortId=XGBOOST-84"> 
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a><a target="_blank" href="https://docs.neptune.ai/integrations/xgboost/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction

This guide will show you how to:

* Initialize Neptune and create a `run`,
* Create a `NeptuneCallback()`,
* Log model training metrics to Neptune using `NeptuneCallback()`.

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
%pip install -U graphviz scikit-learn neptune[xgboost] xgboost

In [ ]:
# To fix the random RuntimeError: main thread is not in main loop error in Windows
import matplotlib.pyplot as plt

plt.switch_backend("agg")

## Import libraries

In [ ]:
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

## Create a run

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    name="xgb-train",
    tags=["xgb-integration", "train"],
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
import neptune

run = neptune.init_run(
    project="common/xgboost-integration",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    name="xgb-train",
    tags=["xgb-integration", "train"],
)

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

## Create NeptuneCallback()

In [ ]:
from neptune.integrations.xgboost import NeptuneCallback

neptune_callback = NeptuneCallback(run=run, log_tree=[0, 1, 2, 3])

This callback will do metadata logging during training. You will pass it to the XGBoost `train()` function.

It also works with `cv()` function and sklearn-like API of the XGBoost (in this case you pass it to the `model.fit()` function).

----

Notice that you will log trees with indices `[0, 1, 2, 3]`.

## Prepare data and define parameters

In [ ]:
# Prepare data
X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_test, label=y_test)

# Define parameters
model_params = {
    "eta": 0.7,
    "gamma": 0.001,
    "max_depth": 9,
    "objective": "reg:squarederror",
    "eval_metric": ["mae", "rmse"],
}
evals = [(dtrain, "train"), (dval, "valid")]
num_round = 57

## Train the model and log metadata to the run in Neptune

In [ ]:
xgb.train(
    params=model_params,
    dtrain=dtrain,
    num_boost_round=num_round,
    evals=evals,
    callbacks=[
        neptune_callback,
        xgb.callback.LearningRateScheduler(lambda epoch: 0.99**epoch),
        xgb.callback.EarlyStopping(rounds=30),
    ],
)

This cell do the model trainig and metadata logging to Neptune.

## Stop logging

<font color=red>**Warning:**</font><br>
Once you are done logging, you should stop tracking the run using the `stop()` method.
This is needed only while logging from a notebook environment. While logging through a script, Neptune automatically stops tracking once the script has completed execution.


In [ ]:
run.stop()

## Analyze logged metadata in the Neptune app

Go to the run link and explore metadata (metrics, all parameters, learning rate, pickled model, visualizations) that were logged to the run in Neptune.

Link should look like this:

https://app.neptune.ai/o/common/org/xgboost-integration/e/XGBOOST-84/all?path=training